# Логичтическая регрессия, метод опорных векторов, one-hot кодирование

### О задании

В этом задании вы:
- настроите метод опорных векторов
- изучите методы работы с категориальными переменными

In [57]:
%pylab inline
import warnings
warnings.simplefilter("ignore")
import pandas as pd
import numpy as np

from sklearn.base import BaseEstimator
from sklearn.model_selection import train_test_split

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


__Задание 1.__ Обучение логистической регрессии на реальных данных и оценка качества классификации.

**(5 баллов)**


Загрузим данные с конкурса [Kaggle Porto Seguro’s Safe Driver Prediction](https://www.kaggle.com/c/porto-seguro-safe-driver-prediction) (вам нужна только обучающая выборка). Задача состоит в определении водителей, которые в ближайший год воспользуются своей автомобильной страховкой (бинарная классификация). Но для нас важна будет не сама задача, а только её данные. При этом под нужды задания мы немного модифицируем датасет.

In [58]:
import pandas as pd
data = pd.read_csv('C:\\Users\\vinik\\Githubik\\ml-homework\\task6_classification\\porto-seguro-safe-driver-prediction\\train.csv', index_col=0)
target = data.target.values
data = data.drop('target', axis=1)

data.head()

,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
id,,,,,,,,,,,,,,,,,,,,,
7,2,2,5,1,0,0,1,0,0,0,...,9,1,5,8,0,1,1,0,0,1
9,1,1,7,0,0,0,0,1,0,0,...,3,1,1,9,0,1,1,0,1,0
13,5,4,9,1,0,0,0,1,0,0,...,4,2,7,7,0,1,1,0,1,0
16,0,1,2,0,0,1,0,0,0,0,...,2,2,4,9,0,0,0,0,0,0
17,0,2,0,1,0,1,0,0,0,0,...,3,1,1,3,0,0,0,1,1,0


Пересемплируем выборку так, чтобы положительных и отрицательных объектов в выборке было одинаковое число. Разделим на обучающую и тестовую выборки.

In [59]:
from sklearn.model_selection import train_test_split
np.random.seed(910)
mask_plus = np.random.choice(np.where(target == 1)[0], 100000, replace=True)
mask_zero = np.random.choice(np.where(target == 0)[0], 100000, replace=True)

data = pd.concat((data.iloc[mask_plus], data.iloc[mask_zero]))
target = np.hstack((target[mask_plus], target[mask_zero]))

target_pd = pd.DataFrame({'target': target})

data = data.reset_index(drop=True)
target_pd = target_pd.reset_index(drop=True)

df = pd.concat([data, target_pd], axis=1)

shuffled_df = df.sample(frac=1, random_state=42).reset_index(drop=True)  # frac=1 means all rows are shuffled

# Split the shuffled list back into data and target

shuffled_data = shuffled_df.drop('target', axis=1).reset_index(drop=True)
shuffled_target = np.array(shuffled_df['target'])

data = shuffled_data
target = shuffled_target

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.5, random_state=46)

Не забудьте отнормировать признаки (можно воспользоваться StandardScaler или сделать это вручную). Пока не будем обращать внимание на то, что некоторые признаки категориальные (этим мы займёмся позже).

In [60]:
from sklearn.preprocessing import StandardScaler

start_data = data.copy()

def normalize_data(data): #dont forget to use this!!!
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(data)
    scaled_data = pd.DataFrame(scaled_data, columns=data.columns)
    return scaled_data

scaled_data = normalize_data(data)
scaled_data.describe()

,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
count,2.000000e+05,2.000000e+05,2.000000e+05,2.000000e+05,2.000000e+05,2.000000e+05,2.000000e+05,2.000000e+05,2.000000e+05,2.000000e+05,...,2.000000e+05,2.000000e+05,2.000000e+05,2.000000e+05,2.000000e+05,2.000000e+05,2.000000e+05,2.000000e+05,2.000000e+05,2.000000e+05
mean,-1.918465e-17,9.514167e-17,1.596234e-16,3.017142e-17,-1.822542e-17,-4.645173e-17,-2.959410e-17,8.718359e-17,-1.006129e-16,7.105427e-20,...,1.184475e-16,-3.794298e-17,-4.554579e-17,-4.089173e-17,7.460699e-19,5.782042e-17,-2.359002e-17,-6.554757e-18,-2.522427e-18,2.739142e-17
std,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,...,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00
min,-9.905914e-01,-3.509020e+00,-1.636898e+00,-2.878048e+00,-1.000223e+00,-7.355743e-01,-6.474524e-01,-4.620329e-01,-4.647157e-01,-2.168458e-02,...,-2.323999e+00,-1.201959e+00,-1.697466e+00,-2.748155e+00,-3.751168e-01,-1.307329e+00,-1.113178e+00,-6.352699e-01,-7.257531e-01,-4.246867e-01
25%,-9.905914e-01,-5.435048e-01,-9.067708e-01,-8.647089e-01,-3.344130e-01,-7.355743e-01,-6.474524e-01,-4.620329e-01,-4.647157e-01,-2.168458e-02,...,-6.163894e-01,-3.694304e-01,-5.160407e-01,-5.625855e-01,-3.751168e-01,-1.307329e+00,-1.113178e+00,-6.352699e-01,-7.257531e-01,-4.246867e-01
50%,-4.948532e-01,-5.435048e-01,-1.766434e-01,-8.647089e-01,-3.344130e-01,-7.355743e-01,-6.474524e-01,-4.620329e-01,-4.647157e-01,-2.168458e-02,...,-1.894870e-01,-3.694304e-01,7.467198e-02,-1.983240e-01,-3.751168e-01,7.649183e-01,8.983289e-01,-6.352699e-01,-7.257531e-01,-4.246867e-01
75%,4.966230e-01,9.392528e-01,9.185477e-01,1.148630e+00,-3.344130e-01,1.359482e+00,1.544515e+00,-4.620329e-01,-4.647157e-01,-2.168458e-02,...,6.643178e-01,4.630982e-01,6.653846e-01,5.301991e-01,-3.751168e-01,7.649183e-01,8.983289e-01,1.574134e+00,1.377879e+00,-4.246867e-01
max,2.479576e+00,3.904768e+00,2.378802e+00,1.148630e+00,3.660446e+00,1.359482e+00,1.544515e+00,2.164348e+00,2.151853e+00,4.611572e+01,...,5.360244e+00,5.458269e+00,5.981798e+00,5.265599e+00,2.665836e+00,7.649183e-01,8.983289e-01,1.574134e+00,1.377879e+00,2.354677e+00


Обучите логистическую регрессию с удобными для вас параметрами, примените регуляризацию. Сделайте предсказание на тестовой части выборки. Посчитайте accuracy, precision, recall и F меру

In [61]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()

X_train, X_test, y_train, y_test = train_test_split(scaled_data, target, test_size=0.5, random_state=46)

model.fit(X_train, y_train)

prediction = model.predict(X_test)

def get_info_prediction(prediction, y_test):
    assert len(prediction) == len(y_test)
    TP = 0
    TN = 0
    FP = 0
    FN = 0

    for i in range(len(prediction)):
        if prediction[i] == 0 and y_test[i] == 0:
            TN += 1
        if prediction[i] == 1 and y_test[i] == 1:
            TP += 1
        if prediction[i] == 0 and y_test[i] == 1:
            FN += 1
        if prediction[i] == 1 and y_test[i] == 0:
            FP += 1

    accuracy = (TN + TP) / (TN + TP + FP + FN)
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)

    F_score = 2 * precision * recall / (precision + recall)

    return accuracy, precision, recall, F_score

accuracy, precision, recall, F_score = get_info_prediction(prediction, y_test)

print("accuracy = ", accuracy, "precision = ", precision, "recall = ", recall, "F_score = ", F_score)
# to-do: write conclusions

accuracy =  0.59075 precision =  0.6002479824237019 recall =  0.5503170994375972 F_score =  0.5741991197860851


__Выводы__ в свободной форме: Без использования какой-то особой нормализации данных с помошью логистической регрессии мы получаем accuracy ~ 59%, precision ~ 60% и recall ~ 55% и F_score ~ 0.574, что неплохо, лучше чем подбрасывание монетки!

## Часть 2. Работа с категориальными переменными

В этой части мы научимся обрабатывать категориальные переменные, так как закодировать их в виде чисел недостаточно (это задаёт некоторый порядок, которого на категориальных переменных может и не быть). Существует два основных способа обработки категориальных значений:
- One-hot-кодирование
- Счётчики (CTR, mean-target кодирование, ...) — каждый категориальный признак заменяется на среднее значение целевой переменной по всем объектам, имеющим одинаковое значение в этом признаке.

Начнём с one-hot-кодирования. Допустим наш категориальный признак $f_j(x)$ принимает значения из множества $C=\{c_1, \dots, c_m\}$. Заменим его на $m$ бинарных признаков $b_1(x), \dots, b_m(x)$, каждый из которых является индикатором одного из возможных категориальных значений:
$$
b_i(x) = [f_j(x) = c_i]
$$

__Задание 1.__ Закодируйте все категориальные признаки с помощью one-hot-кодирования. Обучите логистическую регрессию и посмотрите, как изменилось качество модели (с тем, что было ранее). Измерьте время, потребовавшееся на обучение модели.

__(3 балла)__

In [62]:
# one-hot encoding
import warnings
import time

data = start_data.copy()

def transform_data(data):
    new_data = pd.DataFrame()

    for v in data:
        if v[-3:] == "cat": # переменная категориальная
            categories = dict()
            uniq = set()
            ptr = 0

            for x in data[v]:
                uniq.add(x)

            uniq = sorted(uniq)

            for value in uniq:
                categories[value] = ptr
                ptr += 1

            ptr = 0

            for value in uniq:
                arr = []
                for x in data[v]:
                    if x == value:
                        arr.append(1)
                    else:
                        arr.append(0)
                
                ptr += 1

                new_data[v + "_" + str(value)] = arr
        else:
            new_data[v] = data[v]
    
    return new_data

OneHotTransformed_data = transform_data(data)

OneHotTransformed_data.describe()


,ps_ind_01,ps_ind_02_cat_-1,ps_ind_02_cat_1,ps_ind_02_cat_2,ps_ind_02_cat_3,ps_ind_02_cat_4,ps_ind_03,ps_ind_04_cat_-1,ps_ind_04_cat_0,ps_ind_04_cat_1,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
count,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.00000,200000.000000,...,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000
mean,1.998215,0.001070,0.718275,0.212700,0.047875,0.020080,4.483870,0.000835,0.56884,0.430325,...,5.443865,1.443745,2.873590,7.544455,0.123355,0.630875,0.553405,0.287530,0.345000,0.152800
std,2.017199,0.032693,0.449841,0.409218,0.213502,0.140274,2.739255,0.028884,0.49524,0.495123,...,2.342462,1.201163,1.692875,2.745287,0.328845,0.482569,0.497141,0.452612,0.475369,0.359796
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.00000,0.000000,...,4.000000,1.000000,2.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,4.000000,0.000000,1.00000,0.000000,...,5.000000,1.000000,3.000000,7.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000
75%,3.000000,0.000000,1.000000,0.000000,0.000000,0.000000,7.000000,0.000000,1.00000,1.000000,...,7.000000,2.000000,4.000000,9.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000
max,7.000000,1.000000,1.000000,1.000000,1.000000,1.000000,11.000000,1.000000,1.00000,1.000000,...,18.000000,8.000000,13.000000,22.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [63]:
OneHotTransformed_data = normalize_data(OneHotTransformed_data)

# Итак, у нас есть new_data, обучим logistic regression

X_train, X_test, y_train, y_test = train_test_split(OneHotTransformed_data, target, test_size=0.5, random_state=46)

model = LogisticRegression()

In [64]:
%%time

model.fit(X_train, y_train)

CPU times: total: 2.3 s
Wall time: 1.99 s


LogisticRegression()

In [65]:
prediction = model.predict(X_test)

accuracy, precision, recall, F_score = get_info_prediction(prediction, y_test)

print("accuracy = ", accuracy, "precision = ", precision, "recall = ", recall, "F_score = ", F_score)

accuracy =  0.59666 precision =  0.6051999313540415 recall =  0.5626420964460931 F_score =  0.5831455796936688


Короче я не знал, что есть OneHotEncoder, сейчас попробуем с ним

In [66]:
from sklearn.preprocessing import OneHotEncoder

OneHotEncoder_data = start_data.copy()

encoder = OneHotEncoder(sparse=False)  # You can customize options as needed

categories = []

for v in data:
    if v[-3:] == "cat":
        categories.append(v)

data_categorial = data[categories].copy()

encoded_data = encoder.fit_transform(data_categorial)

encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(categories))

OneHotEncoder_data.drop(columns=categories, inplace=True)  # Remove original categorical columns

OneHotEncoder_data = OneHotEncoder_data.reset_index(drop=True)  # Reset index and drop the old index
encoded_df = encoded_df.reset_index(drop=True)

OneHotEncoder_data = pd.concat([OneHotEncoder_data, encoded_df], axis=1) 

OneHotEncoder_data = normalize_data(OneHotEncoder_data)

X_train, X_test, y_train, y_test = train_test_split(OneHotEncoder_data, target, test_size=0.5, random_state=46)

model = LogisticRegression()

In [67]:
%%time

model.fit(X_train, y_train)

CPU times: total: 2 s
Wall time: 1.99 s


LogisticRegression()

In [68]:
prediction = model.predict(X_test)

accuracy, precision, recall, F_score = get_info_prediction(prediction, y_test)

print("accuracy = ", accuracy, "precision = ", precision, "recall = ", recall, "F_score = ", F_score)

accuracy =  0.59666 precision =  0.6051999313540415 recall =  0.5626420964460931 F_score =  0.5831455796936688


Мини - вывод: OneHotEncoder работает также самописная версия :), но в целом OneHotEncoding улучшило результаты (accuracy улушилась на 0.5%, recall на ~1.2% и F_score на ~0.9%)

Как можно было заменить, one-hot-кодирование может сильно увеличивать количество признаков в датасете, что сказывается на памяти, особенно, если некоторый признак имеет большое количество значений. Эту проблему решает другой способ кодирование категориальных признаков — счётчики. Основная идея в том, что нам важны не сами категории, а значения целевой переменной, которые имеют объекты этой категории. Каждый категориальный признак мы заменим средним значением целевой переменной по всем объектам этой же категории:
$$
g_j(x, X) = \frac{\sum_{i=1}^{l} [f_j(x) = f_j(x_i)][y_i = +1]}{\sum_{i=1}^{l} [f_j(x) = f_j(x_i)]}
$$

__Задание 2.__ Закодируйте категориальные переменные с помощью счётчиков (ровно так, как описано выше без каких-либо хитростей). Обучите логистическую регрессию и посмотрите на качество модели на тестовом множестве. Сравните время обучения с предыдущим экспериментов. Заметили ли вы что-то интересное?

__(2 балла)__

In [69]:
CTR_data = start_data.copy()

for v in start_data:
    if v in categories:
        uniq = np.unique(data[v])
        for value in uniq:
            CTR_data[v][start_data[v] == value] = np.mean(target[start_data[v] == value])

CTR_data.describe()

,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
count,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,...,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000
mean,1.998215,0.500000,4.483870,0.500000,0.500000,0.351100,0.295375,0.175920,0.177605,0.000470,...,5.443865,1.443745,2.873590,7.544455,0.123355,0.630875,0.553405,0.287530,0.345000,0.152800
std,2.017199,0.015959,2.739255,0.020419,0.050252,0.477315,0.456212,0.380753,0.382181,0.021674,...,2.342462,1.201163,1.692875,2.745287,0.328845,0.482569,0.497141,0.452612,0.475369,0.359796
min,0.000000,0.494915,0.000000,0.486218,0.480547,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.494915,2.000000,0.486218,0.480547,0.000000,0.000000,0.000000,0.000000,0.000000,...,4.000000,1.000000,2.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.494915,4.000000,0.486218,0.480547,0.000000,0.000000,0.000000,0.000000,0.000000,...,5.000000,1.000000,3.000000,7.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000
75%,3.000000,0.500783,7.000000,0.517318,0.480547,1.000000,1.000000,0.000000,0.000000,0.000000,...,7.000000,2.000000,4.000000,9.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000
max,7.000000,0.911215,11.000000,0.964072,0.697423,1.000000,1.000000,1.000000,1.000000,1.000000,...,18.000000,8.000000,13.000000,22.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [70]:
CTR_data = normalize_data(CTR_data)

X_train, X_test, y_train, y_test = train_test_split(CTR_data, target, test_size=0.5, random_state=46)

model = LogisticRegression()

In [71]:
%%time

model.fit(X_train, y_train)

CPU times: total: 156 ms
Wall time: 278 ms


LogisticRegression()

In [72]:
prediction = model.predict(X_test)

accuracy, precision, recall, F_score = get_info_prediction(prediction, y_test)

print("accuracy = ", accuracy, "precision = ", precision, "recall = ", recall, "F_score = ", F_score)

accuracy =  0.59449 precision =  0.602155805977462 recall =  0.5637389812931275 F_score =  0.5823144667044342


__Вывод:__ При использовании счетчиков(CTR) и OneHotEncoding мы получаем похожие метрики, отличааются на ~0.1-0.3% (precision лучше у OneHotEncoding, а recall у CTR), но при этом обучение при использовании счетчиков тратит гораздо меньше времени (0.4ms VS 5s), чем OneHotEncoding. Очевидно, что это связано с тем, что при использовании OneHotEncoding количество данных значительно больше, чем при использовании CTR, из-за этого мы и видим разницу в скорости обучения.

Отметим, что такие признаки сами по себе являются классификаторами и, обучаясь на них, мы допускаем "утечку" целевой переменной в признаки. Это ведёт к переобучению, поэтому считать такие признаки необходимо таким образом, чтобы при вычислении для конкретного объекта его целевая метка не использовалась. Это можно делать следующими способами:
- вычислять значение счётчика по всем объектам расположенным выше в датасете (например, если у нас выборка отсортирована по времени)
- вычислять по фолдам, то есть делить выборку на некоторое количество частей и подсчитывать значение признаков по всем фолдам кроме текущего (как делается в кросс-валидации)
- внесение некоторого шума в посчитанные признаки (необходимо соблюсти баланс между избавление от переобучения и полезностью признаков).

__Задание 3.__ Реализуйте корректное вычисление счётчиков двумя из трех вышеперчисленных способов, сравните. Снова обучите логистическую регрессию, оцените качество. Сделайте выводы.

__(3 балла)__

Будем счетчики вычислять по фолдам (разбиваем на cnt_blocks блоков)

In [73]:
fold_data = start_data.copy()

def get_sum (v, index, cumsum, cnt_blocks, n):
    part = n // cnt_blocks + 1
    l = (index // part) * part
    r = min(n, (index // part + 1) * (part))
    length = l + (n - r)
    sum = cumsum[l] + cumsum[n] - cumsum[r - 1]
    return sum / length

for v in categories:
    cnt_blocks = 6
    ans = []

    uniq = np.unique(data[v])
    cumsum = dict()
    cnt = dict()

    for u in uniq:
        np_data = np.array(target[data[v] == u])
        cumsum[u] = np.hstack((np.array([0]), np.cumsum(np_data)))
    
    for value in data[v]:
        if not value in cnt.items():
            cnt[value] = 0
        ans.append(get_sum(value, cnt[value], cumsum[value], cnt_blocks, len(cumsum[value]) - 1))
        cnt[value] += 1

    fold_data[v] = pd.Series(ans, index=data[v].index)

fold_data.describe()

,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
count,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,...,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000
mean,1.998215,0.500325,4.483870,0.500340,0.500336,0.351100,0.295375,0.175920,0.177605,0.000470,...,5.443865,1.443745,2.873590,7.544455,0.123355,0.630875,0.553405,0.287530,0.345000,0.152800
std,2.017199,0.015949,2.739255,0.020522,0.050333,0.477315,0.456212,0.380753,0.382181,0.021674,...,2.342462,1.201163,1.692875,2.745287,0.328845,0.482569,0.497141,0.452612,0.475369,0.359796
min,0.000000,0.495397,0.000000,0.486583,0.480839,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.495397,2.000000,0.486583,0.480839,0.000000,0.000000,0.000000,0.000000,0.000000,...,4.000000,1.000000,2.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.495397,4.000000,0.486583,0.480839,0.000000,0.000000,0.000000,0.000000,0.000000,...,5.000000,1.000000,3.000000,7.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000
75%,3.000000,0.502319,7.000000,0.517610,0.480839,1.000000,1.000000,0.000000,0.000000,0.000000,...,7.000000,2.000000,4.000000,9.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000
max,7.000000,0.915730,11.000000,0.971223,0.698358,1.000000,1.000000,1.000000,1.000000,1.000000,...,18.000000,8.000000,13.000000,22.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [74]:
fold_data = normalize_data(fold_data)

X_train, X_test, y_train, y_test = train_test_split(fold_data, target, test_size=0.5, random_state=46)

model = LogisticRegression()

In [75]:
%%time

model.fit(X_train, y_train)

CPU times: total: 234 ms
Wall time: 288 ms


LogisticRegression()

In [76]:
prediction = model.predict(X_test)

accuracy, precision, recall, F_score = get_info_prediction(prediction, y_test)

print("accuracy = ", accuracy, "precision = ", precision, "recall = ", recall, "F_score = ", F_score)

accuracy =  0.59412 precision =  0.6017985764821208 recall =  0.563200510550038 F_score =  0.581860139283801


Попробуем добавить немного шума счетчикам

In [77]:
CTR_noise_data = start_data.copy()

for v in start_data:
    if v in categories:
        uniq = np.unique(data[v])
        for value in uniq:
            current = np.array(target[start_data[v] == value])
            CTR_noise_data[v][start_data[v] == value] = np.mean(current) + np.random.random(current.shape) * 0.01

CTR_noise_data.describe()

,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
count,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,...,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000
mean,1.998215,0.505002,4.483870,0.504997,0.505009,0.351100,0.295375,0.175920,0.177605,0.000470,...,5.443865,1.443745,2.873590,7.544455,0.123355,0.630875,0.553405,0.287530,0.345000,0.152800
std,2.017199,0.016215,2.739255,0.020622,0.050332,0.477315,0.456212,0.380753,0.382181,0.021674,...,2.342462,1.201163,1.692875,2.745287,0.328845,0.482569,0.497141,0.452612,0.475369,0.359796
min,0.000000,0.494915,0.000000,0.486218,0.480547,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.498400,2.000000,0.490607,0.483476,0.000000,0.000000,0.000000,0.000000,0.000000,...,4.000000,1.000000,2.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.501814,4.000000,0.495000,0.486395,0.000000,0.000000,0.000000,0.000000,0.000000,...,5.000000,1.000000,3.000000,7.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000
75%,3.000000,0.507395,7.000000,0.521512,0.489310,1.000000,1.000000,0.000000,0.000000,0.000000,...,7.000000,2.000000,4.000000,9.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000
max,7.000000,0.921214,11.000000,0.974049,0.707421,1.000000,1.000000,1.000000,1.000000,1.000000,...,18.000000,8.000000,13.000000,22.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [78]:
CTR_noise_data = normalize_data(CTR_noise_data)

X_train, X_test, y_train, y_test = train_test_split(CTR_noise_data, target, test_size=0.5, random_state=46)

model = LogisticRegression()

In [79]:
%%time

model.fit(X_train, y_train)

CPU times: total: 203 ms
Wall time: 310 ms


LogisticRegression()

In [80]:
prediction = model.predict(X_test)

accuracy, precision, recall, F_score = get_info_prediction(prediction, y_test)

print("accuracy = ", accuracy, "precision = ", precision, "recall = ", recall, "F_score = ", F_score)

accuracy =  0.59427 precision =  0.6018607225735028 recall =  0.563798811375693 F_score =  0.5822083552150588


__Вывод:__ Разбиение по фолдам и добавление шума вроде как уменьшает все метрики на ~0-0.3%, но с другой стороны эти приемы дают модели не переобучаться и возможно на реальных данных такая модель будет иметь лучший результат.

## Часть 2. Метод опорных векторов и калибровка вероятностней

__Задание 1.__ Обучение и применение метода опорных векторов.

__(1 балл)__

Обучите метод опорных векторов (воспользуйтесь готовой реализацией LinearSVC из sklearn). Используйте уже загруженные и обработанные в предыдущей части данные.

In [87]:
from sklearn.svm import LinearSVC

svc = LinearSVC(random_state=123)

X_train, X_test, y_train, y_test = train_test_split(CTR_noise_data, target, test_size=0.5, random_state=46)

svc.fit(X_train, y_train)

prediction = svc.predict(X_test)

На той же тестовой части посчитайте все те же метрики. Что вы можете сказать о полученных результатах?

In [88]:
accuracy, precision, recall, F_score = get_info_prediction(prediction, y_test)

print("accuracy = ", accuracy, "precision = ", precision, "recall = ", recall, "F_score = ", F_score)

accuracy =  0.59397 precision =  0.602316850799099 recall =  0.559949742730645 F_score =  0.5803611108240231


__Вывод:__ Метод опорных векторов показывает примерно такие же метрики, как и логистическая регрессия на предыдущих данных, но при этом обучение модели занимает примерно 50 секунд, в отличие от метода счетчиков(0.4 секунды в среднем).

__Report__: Самой лучшей моделью по метрикам оказались просто модель с использованием счетчиков (без ничего). Но остальные модели, кроме обычной логистической регрессии (без изменения данных), имели очень похожие метрики (разница ~0.1-0.3%). 